In [1]:
import sys
!{sys.executable} -m pip install sodapy

You are using pip version 9.0.3, however version 18.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [2]:
import pandas as pd
from sodapy import Socrata

In [3]:
client = Socrata('data.cityofnewyork.us', None)

In [4]:
results = client.get(
    'fhrw-4uyv',
    select='created_date, borough, complaint_type',
    where='''\
        created_date >= "2017-01-01T00:00:00"
        and created_date < "2018-01-01T00:00:00"
        ''',
    limit=500)

In [5]:
df = pd.DataFrame.from_records(results)

In [6]:
df['complaint_type'].value_counts()

Graffiti                                     151
New Tree Request                              58
General Construction/Plumbing                 46
Building/Use                                  43
Damaged Tree                                  42
Dead/Dying Tree                               40
Overgrown Tree/Branches                       16
Street Light Condition                        15
Plumbing                                      13
Water System                                  11
Street Sign - Missing                          9
DCA / DOH New License Application Request      6
Root/Sewer/Sidewalk Condition                  6
DOF Property - Payment Issue                   5
PAINT/PLASTER                                  4
Street Sign - Damaged                          3
Special Projects Inspection Team (SPIT)        3
Sanitation Condition                           2
Street Condition                               2
WATER LEAK                                     2
Sewer               